In [3]:
%cd ..

/Users/juansegundohevia/Documents/repos/TexTract


In [7]:
import pickle as pkl
import numpy as np
import os 
import torch

# Pix2Tex imports
from pix2tex.dataset.dataset import Im2LatexDataset
from pix2tex.utils import in_model_path, parse_args, seed_everything, get_optimizer, get_scheduler, gpu_memory_check
from pix2tex.eval import evaluate
from pix2tex.models import get_model

/Users/juansegundohevia/anaconda3/envs/pix2text/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vanilla train

In [8]:
train_ds = Im2LatexDataset().load("pix2tex/dataset/data/train.pkl")
train_ds

In [10]:
train_ds.images # images attribute are empty

[]

# Handwritten train

In [11]:
hw_train_ds = Im2LatexDataset().load("pix2tex/dataset/data/handwritten_total.pkl")
hw_train_ds

In [15]:
len(hw_train_ds.images) # images attribute are empty

10822

In [16]:
len(hw_train_ds.indices) # images attribute are empty

10822

In [18]:
hw_train_ds.indices[0]

2819

## Split HW images and equations into train-test splits

In [23]:
with open("pix2tex/resources/CROHME/CROHME_math.txt", "r") as f:
    equations = f.readlines()

equations[0]
len(equations)

10846

In [24]:
# make a list with the images fnames
images = os.listdir("pix2tex/resources/CROHME/images")
images[0]

'0002819.png'

In [30]:
len(images)

10822

In [26]:
# to split the HW dataset, I just need to place the images in 3 separate
# folders
train_idx = int(len(hw_train_ds) * .7)
val_idx = train_idx + int(len(hw_train_ds) * .2)
test_idx = val_idx + int(len(hw_train_ds) * .1)

print(train_idx, val_idx, test_idx) # close to total

7575 9739 10821


In [29]:
train_images = images[:train_idx]
val_images = images[train_idx:val_idx]
test_images = images[val_idx:]

print("Split lengths",len(train_images), len(val_images), len(test_images))
print("Total",sum([len(train_images), len(val_images), len(test_images)]))

Split lengths 7575 2164 1083
Total 10822


In [34]:
# move the images to the respective folders
ORIGIN_PATH = "pix2tex/dataset/CROHME/images" # we're taking hte images from here
HW_DATA_PATH = "pix2tex/dataset/handwritten"
os.makedirs(HW_DATA_PATH, exist_ok=True)
os.makedirs(f"{HW_DATA_PATH}/train", exist_ok=True)
os.makedirs(f"{HW_DATA_PATH}/val", exist_ok=True)
os.makedirs(f"{HW_DATA_PATH}/test", exist_ok=True)

import shutil
from tqdm import tqdm

for img in tqdm(train_images):
    shutil.move(os.path.join(ORIGIN_PATH, img), os.path.join(HW_DATA_PATH,"train", img))

for img in tqdm(val_images):
    shutil.move(os.path.join(ORIGIN_PATH, img), os.path.join(HW_DATA_PATH,"val", img))

for img in tqdm(test_images):
    shutil.move(os.path.join(ORIGIN_PATH, img), os.path.join(HW_DATA_PATH,"test", img))

100%|██████████| 1083/1083 [00:00<00:00, 16538.92it/s]


## Check they were split correctly

In [36]:
hw_train = Im2LatexDataset().load("pix2tex/dataset/handwritten/train_hw.pkl")

In [37]:
len(hw_train.images)

7575

In [38]:
len(hw_train.indices)

7575